In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [2]:
# 1. Basic autograd example 1

# create tensor

x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

In [3]:
y = w * x + b

In [4]:
y.backward()

In [5]:
print(x.grad)

tensor(2.)


In [6]:
print(w.grad)

tensor(1.)


In [7]:
print(b.grad)

tensor(1.)


In [8]:
# 2. Basic autograd example 2

# create tensors of shape (10, 3) and (10, 2)
x = torch.randn(10, 3)
y = torch.randn(10, 2)

In [10]:
# build af fully connected layer
linear = nn.Linear(3, 2)

print('w: ', linear.weight)
print('b: ', linear.bias)

w:  Parameter containing:
tensor([[ 0.1725, -0.3195, -0.4593],
        [-0.0911,  0.3769, -0.2447]], requires_grad=True)
b:  Parameter containing:
tensor([-0.1658, -0.1538], requires_grad=True)


In [12]:
# loss function and optimizer

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# forward pass
pred = linear(x)

In [14]:
# compute loss

loss = criterion(pred, y)
print('loss: ', loss.item())

# backward pass

loss.backward()

loss:  0.7699903845787048


In [16]:
# print out the gradients.
print(linear.weight.grad)
print(linear.bias.grad)

tensor([[-0.2285, -0.9043, -0.3686],
        [-0.3730,  0.1177,  0.1884]])
tensor([0.0011, 0.0314])


In [34]:
# 1-step gradient descent
optimizer.step()

In [35]:
# print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print(loss.item())

0.6153327822685242


In [36]:
# 3. Loading data from numpy

# create a numpy array

x = np.array([[1, 2], [3, 4]])

# convert to a tensor
y = torch.from_numpy(x)

# convert back
z = y.numpy()

In [48]:
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

torch.Size([3, 32, 32])
6


In [ ]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [49]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/kevin/.torch/models/resnet18-5c106cde.pth
100.0%


torch.Size([64, 100])


In [51]:
# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))